### A.  Web Scraping the Booking website for 09/1/2024 - 09/5/2024

### 3. New York ###

In [341]:
#Importing all necessary libraries / modules (in random order, but all of them are needed)

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import time, re
import numpy as np
import matplotlib.pyplot as plt
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait


In [342]:
#################
## For New York##
#################
# Create a new instance of a browser
driver = webdriver.Chrome()

# Navigate to the website
url = "https://www.booking.com/"
driver.get(url)

# Define the aria-label attribute for the button
button_aria_label = "Dismiss sign-in info."

# Wait for the button in the pop-up to be clickable
wait = WebDriverWait(driver, 10)
popup_button = wait.until(EC.element_to_be_clickable((By.XPATH, f'//*[@aria-label="{button_aria_label}"]')))

# Click the button
popup_button.click()

# Wait for a moment after dismissing the popup
time.sleep(2)


# Modify the code to search for a location like New York
# Search for the first location: San Diego
# Locate the search input element by its name attribute
search_input = driver.find_element(By.NAME, 'ss')
# Clear any existing value and enter the search query
search_input.clear()
search_input.send_keys("New York")
# Wait for the search results to load (adjust the time if needed)
time.sleep(5)
#need to spend time to wait otherwise the webstie will automate eliminate your serach result for destination
# Simulate pressing the Enter key to submit the search query
search_input.send_keys(Keys.RETURN)


#select check-in date as September 1, 2023
driver.find_element(By.CSS_SELECTOR, "span[aria-label='1 September 2023']").click()

# select check-out date as September 5, 2023
driver.find_element(By.CSS_SELECTOR, "span[aria-label='5 September 2023']").click()

# click the search button
driver.find_element(By.CSS_SELECTOR, "button[type='submit']").click()



In [343]:
url = driver.current_url
type(url)

'https://www.booking.com/searchresults.html?ss=New+York&ssne=New+York&ssne_untouched=New+York&label=gen173nr-1FCAEoggI46AdIM1gEaKQCiAEBmAExuAEHyAEM2AEB6AEB-AECiAIBqAIDuAKJtc6mBsACAdICJDA2ZWRkMTE3LTgxMmYtNDAxOC04ZGEyLWU1NGRjN2ZiYjYyYdgCBeACAQ&aid=304142&lang=en-us&sb=1&src_elem=sb&src=searchresults&dest_id=20088325&dest_type=city&checkin=2023-09-01&checkout=2023-09-05&group_adults=2&no_rooms=1&group_children=0'

In [344]:
from bs4 import BeautifulSoup
import requests
import re
import random
from pymongo import MongoClient

# Add a user_agent header
user_agent = {"user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.55 Safari/537.36"}



url = driver.current_url
# Get the current URL of the driver
url = driver.current_url

# Use a for loop to loop through pages 1 to 5 (25 results per page)
for num in range(0, 125, 25):
    URL = url + "&offset=" + str(num)  # Use the 'url' variable here
    source = requests.get(URL, headers=user_agent)
    soup = BeautifulSoup(source.content, "html.parser")
    
    # Save each page into a single file
    with open("NY_hotel_search_page_" + str(int((num + 25) / 25)) + ".html", "w", encoding='utf-8') as file:
        file.write(str(soup))
    
    wait_time = random.randrange(10, 20)
    print("Done with page number " + str(int((num + 25) / 25)))
    
    # Wait a random time from 10 to 20 seconds before accessing the website again
    time.sleep(wait_time)

print("All done!")


Done with page number 1
Done with page number 2
Done with page number 3
Done with page number 4
Done with page number 5
All done!


In [432]:
from google.cloud import firestore

# Replace with the path to your service account JSON file
service_account_json= "/Users/name/Desktop/Projects/my-project-hotel-database.json" #Need to be Modified 
db = firestore.Client.from_service_account_json(service_account_json)
db

In [433]:
# for New York City Page listing hotel details
list3=[]
#record each hotel id
dat=251

for num in range(1, 6):
    file_path = "/Users/liangzhao/Desktop/projects/NY_hotel_search_page_"+str(num)+".html"
    with open(file_path, "r", encoding="utf-8") as file:
        html_content = file.read()
    #print(html_content)
    # Parse file as a BeautifulSoup object
        soup = BeautifulSoup(html_content, 'html.parser')

    # find all the information for each hotel 
    #each property card will be changed due to the website 
        all_hotel_info = soup.find_all('div', {'class':'c82435a4b8 a178069f51 a6ae3c2b40 a18aeea94d d794b7a0f7 f53e278e95 da89aeb942'})

        for hotel_info in all_hotel_info:            
                #1. name for each hotel
                all_hotel_names = hotel_info.find_all('div', {'class': 'f6431b446c a23c043802'})
                for names in all_hotel_names:
                    hotel_name = names.text
                    #print(hotel_name)
            
            
                #2. links for each hotel
                all_hotel_links = hotel_info.find_all('a', {'class': 'e13098a59f'})             
                for links in all_hotel_links:
                    hotel_link =  links.get('href')
                    hotel_web = requests.get(hotel_link, headers = user_agent)
                    #print(NY_hotel_link)

                    #2-1: hotel address
                    hotel_soup = BeautifulSoup(hotel_web.content, 'html.parser')
                    hotel_add = hotel_soup.find('span', {'class': 'hp_address_subtitle js-hp_address_subtitle jq_tooltip'})
                    hotel_address = hotel_add.text.strip()


                #3. price for 5 nights
                all_hotel_price = hotel_info.find_all('span', {'class':'f6431b446c fbd1d3018c e729ed5ab6'})
                for prices in all_hotel_price:
                    sign, prices = prices.text.split("$", 1)
                    five_night_prices = prices
                    #print(NY_five_night_prices)


                #4. Review scores for each hotel
                all_hotel_scores = hotel_info.find_all('div', {'class': 'a3b8729ab1 d86cee9b25'})
                for scores in all_hotel_scores:
                    if all_hotel_scores:
                        hotel_score = scores.text
                        #print(NY_hotel_score)
                    else:
                        hotel_score="No Score"

                #5. number of reviews for each hotel
                #5. number of reviews for each hotel
                all_hotel_reviews = hotel_info.find_all('div', {'class': 'abf093bdfe f45d8e4c32 d935416c47'})
                if all_hotel_reviews:
                    # Extract the reviews information
                    reviews_text = all_hotel_reviews[0].text.split()
                    total_reviews = reviews_text[0]
                else:
                    total_reviews = "No Reviews" 
                #print("total Review numbers:",NY_total_reviews)
                

                #6. check if breakfast is included
                check_breakfast = hotel_info.find('span', {'class': 'e05969d63d'})
                try:
                    for breakfast in check_breakfast:
                        hotel_breakfast = 1
                except TypeError:
                    hotel_breakfast = 0
                #print("Breakfast:", NY_hotel_breakfast)

                #7. check if hotel allows free cancellation
                check_cancellation = hotel_info.find('div', {'class': 'd506630cf3'})
                try:
                    for free_cancel in check_cancellation:
                        hotel_free_cancel = "Yes"
                except TypeError:
                    hotel_free_cancel = "NO"
                #print("hotel_free_cancel:", NY_hotel_free_cancel)
                
                #8. distance from center
                all_hotel_distance = hotel_info.find_all('span', {'class':'f419a93f12'})
                for distance in all_hotel_distance:
                    if "from" in distance.text:
                        dis, from_center = distance.text.split(" ", 1)
                        dis_from_center = dis
                        (dis_from_center)
                    
                dat+=1

                # store the info for hotels into a dictionary
                 # Create a dictionary for the current hotel
                hotel_dict = {
                    "_id": dat,
                    "Location": "New York",
                    "hotel_name": hotel_name,
                    "hotel_link": hotel_link,
                    "hotel_address": hotel_address,
                    "hotel_distance_from_center": dis_from_center,
                    "five_nights_prices": five_night_prices,
                    "hotel_score": hotel_score,
                    "hotel_total_reviews": total_reviews,
                    "free_breakfast_option": hotel_breakfast,
                    "free_cancellation_option": hotel_free_cancel}


                list3.append(hotel_dict)
                db.collection("Fall_Destination").add(hotel_dict)


   
    

In [434]:
df=pd.DataFrame(list3)
df

,_id,Location,hotel_name,hotel_link,hotel_address,hotel_distance_from_center,five_nights_prices,hotel_score,hotel_total_reviews,free_breakfast_option,free_cancellation_option
0,252,New York,Pestana Park Avenue,https://www.booking.com/hotel/us/pestana-park-...,"23 East 39th Street, Murray Hill, New York, NY...",1.2,"1,081",8.5,984,0,NO
1,253,New York,Sonder at One Platt,https://www.booking.com/hotel/us/sonder-116-jo...,"1 Platt Street, Wall Street - Financial Distri...",4.4,"1,185",7.8,811,0,NO
2,254,New York,Exotic Studio In Midtown East,https://www.booking.com/hotel/us/exotic-studio...,"223 East 65th Street, New York, NY 10065, Unit...",1,"1,432",7.8,No Reviews,0,NO
3,255,New York,Sonder Henri on 24,https://www.booking.com/hotel/us/sonder-henri-...,"37 West 24th Street, Flatiron District, New Yo...",1.8,"1,171",7.7,179,0,NO
4,256,New York,Pod Times Square,https://www.booking.com/hotel/us/pod-times-squ...,"400 West 42nd Street, Hell's Kitchen, New York...",0.9,917,8.1,"16,104",0,NO
...,...,...,...,...,...,...,...,...,...,...,...
122,374,New York,Courtyard New York Manhattan/Times Square,https://www.booking.com/hotel/us/courtyard-new...,"114 West 40th Street, New York, NY 10018, Unit...",1,"1,382",8.2,"1,834",0,NO
123,375,New York,Hilton New York Fashion District,https://www.booking.com/hotel/us/hilton-new-yo...,"152 West 26th Street, Chelsea, New York, NY 10...",1.7,"1,169",7.3,662,0,NO
124,376,New York,Hilton Garden Inn New York Manhattan Midtown East,https://www.booking.com/hotel/us/hilton-garden...,"206 East 52nd Street, Midtown East, New York, ...",1,"1,391",7.5,"1,276",0,NO
125,377,New York,Royalton Park Avenue,https://www.booking.com/hotel/us/royalton-park...,"420 Park Avenue South, NoMad, New York, NY 100...",1.7,"1,553",8.3,"1,359",0,Yes


In [425]:
#data Validation for debugging 

file_path = "/Users/liangzhao/Desktop/projects/NY_hotel_search_page_5.html"
with open(file_path, "r", encoding="utf-8") as file:
    html_content = file.read()
#print(html_content)
# Parse file as a BeautifulSoup object
    soup = BeautifulSoup(html_content, 'html.parser')

# find all the information for each hotel 
#each property card will be changed due to the website 
    all_hotel_info = soup.find_all('div', {'class':'c82435a4b8 a178069f51 a6ae3c2b40 a18aeea94d d794b7a0f7 f53e278e95 da89aeb942'})

    for hotel_info in all_hotel_info:            
            #1. name for each hotel
            all_hotel_names = hotel_info.find_all('div', {'class': 'f6431b446c a23c043802'})
            for names in all_hotel_names:
                NY_hotel_name = names.text
                #print( NY_hotel_name)


            #5. number of reviews for each hotel
            all_hotel_reviews = hotel_info.find_all('div', {'class': 'abf093bdfe f45d8e4c32 d935416c47'})
            if all_hotel_reviews:
                # Extract the reviews information
                reviews_text = all_hotel_reviews[0].text.split()
                total_reviews = reviews_text[0]
            else:
                total_reviews = "No Reviews" 
            print(total_reviews)


4,052
288
2,147
1,483
1,712
391
1
311
1,060
7,088
815
3,739
26
2,102
2,936
2,194
1,493
3,632
606
1,062
1,834
662
1,276
1,359
271
